# **Unificación de datasets de los niveles 1, 2, y 3**

En este notebook se unifican los datasets de los niveles 1, 2 y 3.

Se fija la distribución de cada uno de los niveles, y se unifica el formato pra hacerlo compatible en el finetuning.

También se crean los datsets de train, test y validación, con una distribución 80/10/10, tras haber hecho un shuffle.

Afortunadamente, no hace falta instalar nuevas librerías, lo que evita las incompatibilidades.

In [ ]:
"""
UNIFICADOR FINAL PARA FINE-TUNING T5
Combina niveles 1, 2, 3 y prepara para entrenamiento
"""

import json
import random
import pandas as pd
from collections import Counter
from typing import List, Dict, Tuple
import numpy as np

He aquí la clase que hace la unificación y el split de train, test, y validación. Estos datasets se almacenan en un directorio en Drive en el directorio dataset_t5_final.

In [ ]:
class DatasetUnifier:
    def __init__(self, seed=42):
        self.seed = seed
        random.seed(seed)
        np.random.seed(seed)

    def load_datasets(self, paths: Dict[str, str]) -> Dict[str, List]:
        """Carga datasets de los tres niveles"""
        datasets = {}

        for nivel, path in paths.items():
            print(f"Cargando {nivel} desde {path}...")

            try:
                # Soporte para JSONL y JSON
                if path.endswith('.jsonl'):
                    with open(path, 'r', encoding='utf-8') as f:
                        data = [json.loads(line) for line in f if line.strip()]
                elif path.endswith('.json'):
                    with open(path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        if isinstance(data, dict) and "examples" in data:
                            data = data["examples"]
                else:
                    raise ValueError(f"Formato no soportado: {path}")

                datasets[nivel] = data
                print(f"  ✓ {len(data)} ejemplos cargados")

            except Exception as e:
                print(f"  ✗ Error cargando {nivel}: {e}")
                datasets[nivel] = []

        return datasets

    def unify_format(self, datasets: Dict[str, List]) -> List[Dict]:
        """Unifica formato de todos los niveles"""
        unified = []
        nivel_map = {"nivel1": 1, "nivel2": 2, "nivel3": 3}

        for nivel_name, ejemplos in datasets.items():
            for i, ej in enumerate(ejemplos):
                # Extraer campos (flexible para diferentes estructuras)

                '''
                instruction = ej.get("instruction",
                                   ej.get("pregunta",
                                         ej.get("question", "")))
                '''

                instruction = ej.get("instruction")

                '''
                input_text = ej.get("input",
                                  ej.get("contexto",
                                        ej.get("context", "")))
                '''

                input_text = ej.get("input")


                '''
                output = ej.get("output",
                              ej.get("respuesta",
                                    ej.get("answer", "")))

                '''

                output = output = ej.get("output")



                # Si no hay output, saltar
                if not output:
                    continue

                # Crear ejemplo unificado
                ejemplo_unificado = {
                    "instruction": instruction.strip(),
                    "input": input_text.strip() if input_text else "",
                    "output": output.strip(),
                    "metadata": {
                        **ej.get("metadata", {}),
                        "id": f"{nivel_name}_{i}",
                        "nivel_original": nivel_name,
                        "dificultad": nivel_map.get(nivel_name, 1),
                        "longitud_pregunta": len(instruction),
                        "longitud_respuesta": len(output)
                    }
                }

                unified.append(ejemplo_unificado)

        print(f"\n📊 Total ejemplos unificados: {len(unified)}")
        return unified

    def balance_dataset(self,
                       unified_data: List[Dict],
                       target_dist: Dict[str, float] = None,
                       max_examples: int = 2500) -> List[Dict]:
        """Balancea el dataset según distribución objetivo"""

        if target_dist is None:
            target_dist = {"nivel1": 0.50, "nivel2": 0.35, "nivel3": 0.15}

        # Agrupar por nivel
        by_level = {}
        for ej in unified_data:
            nivel = ej["metadata"]["nivel_original"]
            if nivel not in by_level:
                by_level[nivel] = []
            by_level[nivel].append(ej)

        print("\n📈 DISTRIBUCIÓN ORIGINAL:")
        for nivel, ejemplos in by_level.items():
            print(f"  {nivel}: {len(ejemplos)} ejemplos")

        # Calcular objetivos
        balanced = []
        for nivel, ratio in target_dist.items():
            target_count = int(max_examples * ratio)
            disponibles = by_level.get(nivel, [])

            if len(disponibles) >= target_count:
                selected = random.sample(disponibles, target_count)
            else:
                selected = disponibles
                print(f"⚠️  {nivel}: Solo {len(disponibles)} disponibles "
                      f"(objetivo: {target_count})")

            balanced.extend(selected)

        # Mezclar
        random.shuffle(balanced)

        print(f"\n🎯 Dataset balanceado: {len(balanced)} ejemplos")
        distrib_final = Counter([e["metadata"]["nivel_original"] for e in balanced])
        for nivel, count in distrib_final.items():
            print(f"  {nivel}: {count} ejemplos ({count/len(balanced)*100:.1f}%)")

        return balanced

    def prepare_for_t5(self,
                      balanced_data: List[Dict],
                      prompt_template: str = None) -> List[Dict]:
        """Prepara datos para fine-tuning de T5"""

        if prompt_template is None:
            prompt_template = (
                "### Instrucción:\n{instruction}\n\n"
                "### Contexto:\n{input}\n\n"
                "### Respuesta:\n"
            )

        t5_data = []

        for ej in balanced_data:
            # Construir input
            input_text = prompt_template.format(
                instruction=ej["instruction"],
                input=ej["input"] if ej["input"] else "Datos electorales españoles"
            )

            # Target
            target_text = ej["output"]

            t5_example = {
                "input_text": input_text.strip(),
                "target_text": target_text.strip(),
                "metadata": ej["metadata"]
            }

            t5_data.append(t5_example)

        # Estadísticas de longitud
        input_lens = [len(e["input_text"]) for e in t5_data]
        target_lens = [len(e["target_text"]) for e in t5_data]

        print(f"\n📏 ESTADÍSTICAS LONGITUD:")
        print(f"  Inputs: avg={np.mean(input_lens):.0f}, "
              f"max={max(input_lens)}, min={min(input_lens)}")
        print(f"  Targets: avg={np.mean(target_lens):.0f}, "
              f"max={max(target_lens)}, min={min(target_lens)}")

        return t5_data

    def create_splits(self,
                     t5_data: List[Dict],
                     splits: Dict[str, float] = None) -> Dict[str, List]:
        """Crea splits train/val/test estratificados"""

        if splits is None:
            splits = {"train": 0.8, "val": 0.1, "test": 0.1}

        # Estratificar por nivel
        by_difficulty = {}
        for ej in t5_data:
            diff = ej["metadata"]["dificultad"]
            if diff not in by_difficulty:
                by_difficulty[diff] = []
            by_difficulty[diff].append(ej)

        # Mezclar dentro de cada nivel
        for diff in by_difficulty:
            random.shuffle(by_difficulty[diff])

        # Crear splits
        result = {split: [] for split in splits.keys()}

        for diff, ejemplos in by_difficulty.items():
            n_total = len(ejemplos)
            start_idx = 0

            for split_name, split_ratio in splits.items():
                n_split = int(n_total * split_ratio)
                end_idx = start_idx + n_split

                result[split_name].extend(ejemplos[start_idx:end_idx])
                start_idx = end_idx

        # Mezclar cada split
        for split_name in result:
            random.shuffle(result[split_name])

        print(f"\n🔀 SPLITS CREADOS:")
        for split_name, split_data in result.items():
            print(f"\n  {split_name.upper()} ({len(split_data)} ejemplos):")

            # Distribución por nivel
            dist = Counter([e["metadata"]["dificultad"] for e in split_data])
            for diff in sorted(dist.keys()):
                count = dist[diff]
                print(f"    Nivel {diff}: {count} ({count/len(split_data)*100:.1f}%)")

        return result

    def save_datasets(self,
                     splits: Dict[str, List],
                     output_dir: str = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_final"):
        """Guarda los splits en formato Hugging Face"""

        import os
        os.makedirs(output_dir, exist_ok=True)

        # Guardar en múltiples formatos
        for split_name, split_data in splits.items():
            # JSONL (para Hugging Face)
            jsonl_path = f"{output_dir}/{split_name}.jsonl"
            with open(jsonl_path, 'w', encoding='utf-8') as f:
                for ejemplo in split_data:
                    f.write(json.dumps(ejemplo, ensure_ascii=False) + '\n')

            # CSV (para inspección)
            csv_path = f"{output_dir}/{split_name}.csv"
            df = pd.DataFrame([
                {
                    "input": e["input_text"],
                    "target": e["target_text"],
                    "dificultad": e["metadata"]["dificultad"],
                    "nivel": e["metadata"]["nivel_original"]
                }
                for e in split_data
            ])
            df.to_csv(csv_path, index=False, encoding='utf-8')

            print(f"  💾 {split_name}: {len(split_data)} ejemplos -> {jsonl_path}")

        # Guardar metadata
        metadata = {
            "total_examples": sum(len(s) for s in splits.values()),
            "split_sizes": {k: len(v) for k, v in splits.items()},
            "creation_date": pd.Timestamp.now().isoformat(),
            "seed": self.seed,
            "description": "Dataset electoral español para fine-tuning T5"
        }

        with open(f"{output_dir}/metadata.json", 'w') as f:
            json.dump(metadata, f, indent=2)

        print(f"\n✅ Dataset guardado en: {output_dir}")

        return output_dir

Esta es la función main(), en el que declaramos los paths a los datasets de los niveles, y que activa los métodos de la clase que realiza la unificación.

In [ ]:
def main():
    """Pipeline completo de unificación"""

    # CONFIGURACIÓN
    PATHS = {
        "nivel1": "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel1.jsonl",
        "nivel2": "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel2.jsonl",
        "nivel3": "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_nivel3.jsonl"
    }

    # 1. Inicializar unificador
    unifier = DatasetUnifier(seed=42)

    # 2. Cargar datasets
    datasets = unifier.load_datasets(PATHS)

    # 3. Unificar formato
    unified = unifier.unify_format(datasets)

    # 4. Balancear (2000 ejemplos total)
    balanced = unifier.balance_dataset(
        unified,
        target_dist={"nivel1": 0.50, "nivel2": 0.35, "nivel3": 0.15},
        max_examples=2000
    )

    # 5. Preparar para T5
    t5_data = unifier.prepare_for_t5(balanced)

    # 6. Crear splits
    splits = unifier.create_splits(
        t5_data,
        splits={"train": 0.8, "val": 0.1, "test": 0.1}
    )

    # 7. Guardar
    output_dir = unifier.save_datasets(splits, "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_t5_final")

    print(f"\n🎉 PROCESO COMPLETADO!")
    print(f"📁 Dataset listo para fine-tuning en: {output_dir}")

    return splits


Comenzamos la unificación, y vemos que se realiza correctamente.

In [ ]:
if __name__ == "__main__":
    splits = main()

Cargando nivel1 desde /content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel1.jsonl...
  ✓ 2000 ejemplos cargados
Cargando nivel2 desde /content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel2.jsonl...
  ✓ 600 ejemplos cargados
Cargando nivel3 desde /content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_nivel3.jsonl...
  ✓ 200 ejemplos cargados

📊 Total ejemplos unificados: 2800

📈 DISTRIBUCIÓN ORIGINAL:
  nivel1: 2000 ejemplos
  nivel2: 600 ejemplos
  nivel3: 200 ejemplos
⚠️  nivel2: Solo 600 disponibles (objetivo: 700)
⚠️  nivel3: Solo 200 disponibles (objetivo: 300)

🎯 Dataset balanceado: 1800 ejemplos
  nivel2: 600 ejemplos (33.3%)
  nivel3: 200 ejemplos (11.1%)
  nivel1: 1000 ejemplos (55.6%)

📏 ESTADÍSTICAS LONGITUD:
  Inputs: avg=246, max=1064, min=141
  Targets: avg=507, max=5182, min=65

🔀 SPLITS CREADOS:

  TRAIN (1440 ejemplos):
    Nivel 1: 800 (55.6%)
    Nivel 2: 480 (33.3%)
    Nivel 3: 160 (11.1%)

  VAL (180 ejemplos):
    Niv